# Dataset
https://www.kaggle.com/mrisdal/fake-news#fake.csv

Example domain classifications (in flux) include:

  * **Fake News**: Sources that fabricate stories out of whole cloth with the intent of pranking the public.
  * **Satire**: Sources that provide humorous commentary on current events in the form of fake news.
  *  **Extreme Bias**: Sources that traffic in political propaganda and gross distortions of fact.
  *  **Conspiracy Theory**: Sources that are well-known promoters of kooky conspiracy theories.
  *  **Rumor Mill**: Sources that traffic in rumors, innuendo, and unverified claims.
  *  **State News**: Sources in repressive states operating under government sanction.
  *  **Junk Science**: Sources that promote pseudoscience, metaphysics, naturalistic fallacies, and other scientifically dubious claims.
  *  **Hate Group**: Sources that actively promote racism, misogyny, homophobia, and other forms of discrimination.
  *  **Clickbait:** Sources that are aimed at generating online advertising revenue and rely on sensationalist headlines or eye-catching pictures.
  *  **Proceed With Caution**: Sources that may be reliable but whose contents require further verification.
  *  The labels of **bs** and **junksci**, etc. do not constitute capital "t" Truth



In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
#import mlutils
%matplotlib inline
import pandas as pd
import tensorflow_hub as hub 
import re
print(tf.__version__)
#print("The Python version is %s.%s.%s" % sys.version_info[:3])


D:\Programas\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.10.0


## Descarga, arregla e inspecciona el dataset Fake news

In [6]:
import os
!mkdir -p dataFake
if not os.path.exists("dataFake/fake.csv"):
    !wget -c -nc https://github.com/chdorrego/tf/raw/master/fake-news.zip -P dataFake
    !unzip dataFake/fake-news.zip -d dataFake
    with open("dataFake/fake.csv", "r") as f:
        lines = f.readlines()
        lines = [i for i in lines if len(i.replace("\n", ""))>0]
    with open("dataFake/fake.csv", "w") as f:
        f.writelines(lines) 

Ya existe el subdirectorio o el archivo -p.
Error mientras se procesaba: -p.
Ya existe el subdirectorio o el archivo dataFake.
Error mientras se procesaba: dataFake.


UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 2087: character maps to <undefined>

In [8]:
!head -5 dataFake/fake.csv

"gc" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [10]:
CSV_COLUMNS  = ['uuid','ord_in_thread','author','published','title','text','language','crawled','site_url','country','domain_rank','thread_title','spam_score','main_img_url','replies_count','participants_count','likes','comments','shares','type_label']
#CSV_DEFAULTS = [[''],[0],[''],[''],[''],[''],[''],[''],[''],[''],[0],[''],[-1],[''],[-1],[-1],[-1],[-1],[-1],['']]
CSV_DEFAULTS = [[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[0]]


type_label =  ['bias','conspiracy','hate','state','junksci','satire','fake']
no_valid_labels = ['bs']

fake= pd.read_csv("dataFake/fake.csv", names = CSV_COLUMNS,skiprows=1)
print(fake.shape)
fake.head()

(12999, 20)


,uuid,ord_in_thread,author,published,title,text,language,crawled,site_url,country,domain_rank,thread_title,spam_score,main_img_url,replies_count,participants_count,likes,comments,shares,type_label
0,6a175f46bcd24d39b3e962ad0f29936721db70db,0,Barracuda Brigade,2016-10-26T21:41:00.000+03:00,Muslims BUSTED: They Stole Millions In Gov’t B...,Print They should pay all the back all the mon...,english,2016-10-27T01:49:27.168+03:00,100percentfedup.com,US,25689.0,Muslims BUSTED: They Stole Millions In Gov’t B...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
1,2bdc29d12605ef9cf3f09f9875040a7113be5d5b,0,reasoning with facts,2016-10-29T08:47:11.259+03:00,Re: Why Did Attorney General Loretta Lynch Ple...,Why Did Attorney General Loretta Lynch Plead T...,english,2016-10-29T08:47:11.259+03:00,100percentfedup.com,US,25689.0,Re: Why Did Attorney General Loretta Lynch Ple...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
2,c70e149fdd53de5e61c29281100b9de0ed268bc3,0,Barracuda Brigade,2016-10-31T01:41:49.479+02:00,BREAKING: Weiner Cooperating With FBI On Hilla...,Red State : \nFox News Sunday reported this mo...,english,2016-10-31T01:41:49.479+02:00,100percentfedup.com,US,25689.0,BREAKING: Weiner Cooperating With FBI On Hilla...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
3,7cf7c15731ac2a116dd7f629bd57ea468ed70284,0,Fed Up,2016-11-01T05:22:00.000+02:00,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,Email Kayla Mueller was a prisoner and torture...,english,2016-11-01T15:46:26.304+02:00,100percentfedup.com,US,25689.0,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,0.068,http://100percentfedup.com/wp-content/uploads/...,0,0,0,0,0,bias
4,0206b54719c7e241ffe0ad4315b808290dbe6c0f,0,Fed Up,2016-11-01T21:56:00.000+02:00,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,Email HEALTHCARE REFORM TO MAKE AMERICA GREAT ...,english,2016-11-01T23:59:42.266+02:00,100percentfedup.com,US,25689.0,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,0.865,http://100percentfedup.com/wp-content/uploads/...,0,0,0,0,0,bias


### Labels

* bs            10919 
* conspiracy      412 
- bias            354 
- hate            246 
- state           121 
- junksci         102 
- satire          100 
- fake             19


## Realizamos un preprocesamiento de los datos 
- Eliminiación de la clase bs debido a la definición del problema
- Se eliminan los caracteres especiales
- Se transforma el valor del target por una equivalencia númerica para facilidad del procesamiento
- Realizamos la división de la data en archivos test y train 


In [28]:
d = fake.loc[fake['type_label'].isin(type_label)]
d['title'].str.replace('\W', ' ')
d['text'].str.replace('\W', ' ')
d['thread_title'].str.replace('\W', '')

d['type_label'].astype("category", categories=type_label).cat.codes


msk = np.random.rand(len(d)) < 0.8

d_train = d[msk==True]
d_test =  d[msk==False]

d_train.to_csv('dataFake/fake_train.csv', sep=';', encoding='utf-8', header=False)
d_test.to_csv('dataFake/fake_test.csv', sep=';', encoding='utf-8', header=False)
print(d_train.count())
print(d_test.count())

D:\Programas\anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  


uuid                  1208
ord_in_thread         1208
author                 822
published             1208
title                 1124
text                  1172
language              1208
crawled               1208
site_url              1208
country               1208
domain_rank           1060
thread_title          1208
spam_score            1208
main_img_url           956
replies_count         1208
participants_count    1208
likes                 1208
comments              1208
shares                1208
type_label            1208
dtype: int64
uuid                  299
ord_in_thread         299
author                206
published             299
title                 276
text                  289
language              299
crawled               299
site_url              299
country               299
domain_rank           259
thread_title          299
spam_score            299
main_img_url          237
replies_count         299
participants_count    299
likes                 299
comme

## Creamos la funcion para la generacion del `input pipeline`

In [2]:
CSV_COLUMNS  = ['pd_rowid','uuid','ord_in_thread','author','published','title','text','language','crawled','site_url','country','domain_rank','thread_title','spam_score','main_img_url','replies_count','participants_count','likes','comments','shares','type_label']
CSV_DEFAULTS = [[0],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[0]]
type_label =  ['bias','conspiracy','hate','state','junksci','satire','fake']


def parse_csv(rows_string_tensor, use_features):
    columns = tf.decode_csv(rows_string_tensor, record_defaults=CSV_DEFAULTS, field_delim=";")
    features = dict(zip(CSV_COLUMNS, columns))   
    
    label = features.pop("type_label")
      
    #label = tf.one_hot(indices=label,depth=len(type_label),dtype=tf.int32)    
    
    return {k:v for k,v in features.items() if k in use_features}, label



def get_dataset(filename, repeat, batch_size, shuffle):
    use_features  = ['title','text','type_label']
    
    dataset = tf.data.TextLineDataset(filename)
    dataset = dataset.repeat(repeat)
    if shuffle:
        dataset = dataset.shuffle(buffer_size=300)
    dataset = dataset.map(lambda x: parse_csv(x, use_features))    
    dataset = dataset.batch(batch_size)
    return dataset



def get_features_defs():
  
    title = hub.text_embedding_column(
        key='title', 
        module_spec='https://tfhub.dev/google/nnlm-en-dim128/1')
    
    text = hub.text_embedding_column(
        key='text', 
        module_spec='https://tfhub.dev/google/nnlm-en-dim128/1')
    
    return [title,text] 
  
  
#Este metodo no se usa en el premade - solo se utiliza para realiza una prueba
def make_data_iterator(filename, repeat, batch_size, shuffle=True):
    
    dataset = get_dataset(filename, repeat, batch_size, shuffle)
    
    features_defs = get_features_defs()
    
    features, label = dataset.make_one_shot_iterator().get_next()
    
    next_X = tf.feature_column.input_layer(features, features_defs)
    next_y = label
    
    return next_X, next_y

Prueba de la carga del dataset 

In [3]:
dataset = tf.data.TextLineDataset("dataFake/fake_train.csv")
dataset = dataset.batch(10)
features = dataset.make_one_shot_iterator().get_next()
with tf.Session() as sess:
    print(sess.run(features))


[b'0;6a175f46bcd24d39b3e962ad0f29936721db70db;0;Barracuda Brigade;2016-10-26T21:41:00.000+03:00;Muslims BUSTED  They Stole Millions In Gov   t Benefits;Print They should pay all the back all the money plus interest  The entire family and everyone who came in with them need to be deported asap  Why did it take two years to bust them   Here we go again    another group stealing from the government and taxpayers  A group of Somalis stole over four million in government benefits over just 10 months   We   ve reported on numerous cases like this one where the Muslim refugees immigrants commit fraud by scamming our system   It   s way out of control  More Related;english;2016-10-27T01:49:27.168+03:00;100percentfedup.com;US;25689.0;MuslimsBUSTEDTheyStoleMillionsInGovtBenefits;0.0;http://bb4sp.com/wp-content/uploads/2016/10/Fullscreen-capture-10262016-83501-AM.bmp.jpg;0;1;0;0;0;0'
 b'1;2bdc29d12605ef9cf3f09f9875040a7113be5d5b;0;reasoning with facts;2016-10-29T08:47:11.259+03:00;Re  Why Did Att

Realizamos una preuba sobre el metodo parse_csv

In [4]:
dataset = tf.data.TextLineDataset("dataFake/fake_train.csv")
dataset = dataset.batch(1)
use_features  = ['title','text','type_label']
dataset = dataset.map(lambda x: parse_csv(x, use_features))     
features = dataset.make_one_shot_iterator().get_next()
with tf.Session() as sess:
    print(sess.run(features))

({'title': array([b'Muslims BUSTED  They Stole Millions In Gov   t Benefits'],
      dtype=object), 'text': array([b'Print They should pay all the back all the money plus interest  The entire family and everyone who came in with them need to be deported asap  Why did it take two years to bust them   Here we go again    another group stealing from the government and taxpayers  A group of Somalis stole over four million in government benefits over just 10 months   We   ve reported on numerous cases like this one where the Muslim refugees immigrants commit fraud by scamming our system   It   s way out of control  More Related'],
      dtype=object)}, array([0]))


*Realizamos* la prueba sobre el embebido del texto utlizando make_data iterator, esto toma más tiempo la primera vez dado que debe descargar el embebido pre-establecido

In [5]:
tf.reset_default_graph()

features, label = make_data_iterator("dataFake/fake_train.csv", repeat=1, batch_size=1, shuffle=False)
tf.logging.set_verbosity(tf.logging.INFO)
#tf.logging.set_verbosity(v)

with tf.Session() as sess:
    tf.tables_initializer().run()
    tf.global_variables_initializer().run() #Linea necesaria para la correcta ejecucion
    f,l = sess.run([features, label])
print(f)
print(l)

INFO:tensorflow:Using C:\Users\cdorrego\AppData\Local\Temp\tfhub_modules to cache modules.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
[[ 1.11478150e+00  8.39062557e-02  2.26131335e-01  1.61706522e-01
   1.03297129e-01  1.80786606e-02  1.07401036e-01 -8.00728053e-02
  -2.36009434e-01  1.25555947e-01  5.09913675e-02  3.04798074e-02
  -1.72189400e-01  1.07544601e-01 -1.61057487e-01  3.63296270e-03
  -3.56842130e-01 -2.04037189e-01 -2.12922499e-01  8.68280113e-01
  -8.55391398e-02  1.18203647e-01 -9.32149142e-02  4.22991738e-02
  -1.84482820e-02 -3.75717908e-01  1.71205357e-01  3.17466492e-03
  -1.46537095e-01 -8.42622444e-02 -1.37651134e-02 -3.95636149e-02
   8.09066892e-02 -1.1770

In [6]:
f.size

256

# Usamos un estimador predefinido  (premade estimator)

In [7]:
classifier = tf.estimator.DNNClassifier(hidden_units=[100,50,10], 
                                        activation_fn = tf.nn.relu,
                                        feature_columns=get_features_defs(),                                        
                                        n_classes = len(type_label),
                                        model_dir="models")

tf.logging.set_verbosity(tf.logging.INFO)
#!rm -rf models
train_input_fn = lambda: get_dataset("dataFake/fake_train.csv", repeat=2, batch_size=10, shuffle=True)

classifier.train(train_input_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'models', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002511B67BB00>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not

In [ ]:
test_input_fn = lambda: get_dataset("dataFake/fake_test.csv", repeat=1, batch_size=10, shuffle=False)
result = classifier.evaluate(test_input_fn)


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-09-04-22:09:45
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from models\model.ckpt-482


In [14]:
print(result)

{'accuracy': 0.6743421, 'average_loss': 0.9403539, 'loss': 9.221535, 'global_step': 482}


In [10]:
history_dict = history.history
history_dict.keys()

NameError: ignored

In [0]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

# "bo" is for "blue dot"
plt.plot(epochs, loss, 'bo', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()